#  Data Acquisition Reporting

### Introduction

Ce notebook constitue le rapport du module Data Acquisition. Ce rapport fait donc office d'explication et de compréhension. Il permet nottament d'expliquer :
- les concepts mis en place
- le role de chaque fonction
- les interactions entre les fonctions


NB : Le code présenté ne constitue pas le code source des module. C'est un code simplifié qui facilite la compréhension. Les fonctionnalitées non présentes dans ce rapport seront expliquées et documéntées.

##### uncomment the line below to download the requierement for this notebook

Elasticsearch6 package is used but you can also use elastiscsearch (that will be work)

In [1]:
"""
!pip install elasticsearch6
!pip install pandas
"""

'\n!pip install elasticsearch6\n!pip install pandas\n'

#### Setup variables

In [2]:
import os

In [3]:
host='localhost'
port = 9200
path_data= f'{os.getcwd()}/data'

if you can't etablish a connexion with the cluster you can use an exemple pickle for the raw_data. that you cant specify the path here :

In [ ]:
path_raw_data = ''

create a directory for the data that will be save

In [4]:
def create_directory(path_directory):
    if not os.path.exists(path_directory):
        os.mkdir(path_directory)

In [5]:
create_directory(path_data)
path_data

'/home/eguimard/PycharmProjects/DataStudy/TestDev/Notebook/data'

### Connecting To Cluster

Connect To ElasticSearch Cluster with python-api

In [6]:
from elasticsearch6 import Elasticsearch

In [7]:
client = Elasticsearch(f'{host}:{port}')

#### Index  used for the search :

In [8]:
index= "events-20220502"

### Make a query  on the Elastic Cluster

here it's a json elastic query wrote in dict python. With this query, we recover all the data where category == 'availability' in the index "events-20220502". 
Here we chose perimeter availability for the report, but it's pretty much the same process for the others perimeters

In [9]:
perimeter ="availability"
VM ="VM1"

In [10]:
query_in_dict ={
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "category": perimeter
          }
        },
        {
          "match": {
            "hostname": VM
          }
        }
      ],
      "must_not": [],
      "should": []
    }
  },
  "from": 0,
  "size": 1000,
  "sort": [],
  "aggs": {}
}

Here we run the query on the cluster. <br>The output correpond to the return of the query. We will name this raw_data because the data are not sorted and there is a lot of useless data

In [ ]:
## save raw _data into pickle for the notebook exemple

In [11]:
try:
    raw_data = client.search(index=index, body=query_in_dict)
except ConnectionError:
    raw_data = pickle.load(path_data)

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc136d337f0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc136d337f0>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
raw_data

raw data is a set of dictionaries and nested lists. So we have to search for the essential information by browsing them 

### Extracting

Here we will exctract the relevant data drom raw_data for futures analysis. The relevant data are twice : 
- the system data accesible at raw_data['hits']['hits'][n]['_source']['details']['system']
- a list of wanted feature that we can define. Features are accesible at raw_data['hits']['hits'][n]['_source']

Here we define a list of wanted feature to recover. In our case we need 'utctimestamp' for the data visualisation with time on the x-axis

In [ ]:
list_wanted_features = ['utctimestamp']

the main function of the data extracting this function return a dict of lists. Each list correspond to a observation like cpu, memory... 

In [ ]:
def exctract_relevant_data_from_raw_data(raw_data, list_wanted_features):
    
    dict_of_list_dict_data = dict()
    
    for results in raw_data['hits']['hits']:
        
        # extract system data recursively and append to dict_final
        system_data = results['_source']['details']['system']
        system_name = list(system_data.keys())[0]
        
        dict_extracted_data = develop_data_recursively_for_extraction(system_data[system_name])
        
        # wanted_features extraction
        dict_extracted_data = extract_wanted_features(results['_source'], list_wanted_features, dict_extracted_data)

        
        # append dict_extracted_data to the corresponding list
        if system_name not in dict_of_list_dict_data:
            dict_of_list_dict_data[system_name] = list()
        dict_of_list_dict_data[system_name].append(dict_extracted_data)
        
    return dict_of_list_dict_data
    

this function  goes back recursively through dict of dict and create feature name foreach value found

In [ ]:
def develop_data_recursively_for_extraction(current_dict, name_feature=''):

    dict_final = dict()

    if type(current_dict) is dict:
        for key, new_obj in current_dict.items():
            dict_final.update(develop_data_recursively_for_extraction(new_obj, f'{name_feature}{key}_'))
            # update can delete field. in theory, we don't have twice the same field
    else:
        dict_final[name_feature[:-1]] = current_dict

    return dict_final

this function allows to extract feature from a list of name : 

In [ ]:
def extract_wanted_features(current_data, list_wanted_features, dict_extracted_data):

    for wanted_feature in list_wanted_features:
        dict_extracted_data[wanted_feature] = current_data[wanted_feature]

    return dict_extracted_data

In [ ]:
dict_list_data = exctract_relevant_data_from_raw_data(raw_data, list_wanted_features)

In output we have a dictionnary of list. We will convert these lists to Dataframe : 

In [ ]:
import pandas as pd

In [ ]:
def transform_list_data_to_dataframe(dict_of_list_data):
    dict_df = dict()

    for key, list_data in dict_of_list_data.items():
        dict_df[key] = pd.DataFrame(list_data)

    return dict_df

In [ ]:
dict_dataFrame = transform_list_data_to_dataframe(dict_list_data)

The name of the dataframes created are the keys of dictionnary : 

In [ ]:
dict_dataFrame.keys()

Each DataFrame is related to an observation like memory, cpu, network...

Finnaly we Save the dict dataframe to the path data define at the top of this notebook. The dataframes stored will be use in the Notebook Data Analyse : 

In [ ]:
def store_dict_df_into_directory(dict_df, save_path_directory, extension):

    for name_df in dict_df.keys():
        try:
            dict_df[name_df].to_csv(f'{save_path_directory}/{name_df}.{extension}', index=False)
        except Exception as err:  # precisez l'exception
            return str(err)

In [ ]:
store_dict_df_into_directory(dict_dataFrame, path_data, extension='csv')

 a display of the dataframe cpu :

In [ ]:
dict_df['cpu']